In [63]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import transform
from functools import partial
import pyproj

from unidecode import unidecode
import re

# Prepare GPS data for each district

In [82]:
# Load the data from a JSON file
with open('large_files/gadm41_VNM_2.json', encoding='utf-8') as file:
    gadm = json.load(file)

gps = pd.json_normalize(gadm['features'])

In [84]:
# Define the function to calculate area and centroid
def calculate_geometric_properties(coords):
    # Convert the coordinates into a Shapely Polygon
    poly = shape({"type": "Polygon", "coordinates": coords[0]})
    
    # Determine the UTM zone number from the centroid longitude
    utm_zone = int((poly.centroid.x + 180) / 6) + 1
    
    # Create the projection transformation
    proj = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),
        pyproj.Proj(proj='utm', zone=utm_zone, datum='WGS84')
    )
    
    # Perform the transformation
    projected_poly = transform(proj, poly)
    
    # Calculate the area in square meters
    area = projected_poly.area
    
    # Get the centroid of the projected polygon
    centroid = projected_poly.centroid
    
    # Convert the centroid back to lat/long
    proj_inverse = partial(
        pyproj.transform,
        pyproj.Proj(proj='utm', zone=utm_zone, datum='WGS84'),
        pyproj.Proj(init='epsg:4326')
    )
    centroid_latlong = transform(proj_inverse, centroid)
    
    # Return the calculated area and centroid coordinates
    return pd.Series({
        'area': area,
        'centroid_lon': centroid_latlong.x,
        'centroid_lat': centroid_latlong.y,
    })

# Apply the function to each row
geometric_properties = gps['geometry.coordinates'].apply(calculate_geometric_properties)

# Join the results to the original DataFrame
gps = gps.join(geometric_properties)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be 

In [85]:
gps

,type,properties.GID_2,properties.GID_0,properties.COUNTRY,properties.GID_1,properties.NAME_1,properties.NL_NAME_1,properties.NAME_2,properties.VARNAME_2,properties.NL_NAME_2,properties.TYPE_2,properties.ENGTYPE_2,properties.CC_2,properties.HASC_2,geometry.type,geometry.coordinates,area,centroid_lon,centroid_lat
0,Feature,VNM.1.1_1,VNM,Vietnam,VNM.1_1,AnGiang,NA,AnPhú,AnPhu,NA,Huyện,District,NA,VN.TT.AL,MultiPolygon,"[[[[105.1566, 10.8287], [105.1601, 10.7814], [...",2.268493e+08,105.100750,10.846396
1,Feature,VNM.1.3_1,VNM,Vietnam,VNM.1_1,AnGiang,NA,ChâuĐốc,ChauDoc,NA,Thànhphố,City,NA,VN.KG.AB,MultiPolygon,"[[[[105.105, 10.6383], [105.0815, 10.6146], [1...",1.033888e+08,105.087417,10.674122
2,Feature,VNM.1.4_1,VNM,Vietnam,VNM.1_1,AnGiang,NA,ChâuPhú,ChauPhu,NA,Huyện,District,NA,VN.HP.AD,MultiPolygon,"[[[[105.1879, 10.4603], [105.1798, 10.4597], [...",4.504320e+08,105.179787,10.557599
3,Feature,VNM.1.5_1,VNM,Vietnam,VNM.1_1,AnGiang,NA,ChâuThành,ChauThanh,NA,Huyện,District,NA,VN.GL.AK,MultiPolygon,"[[[[105.3709, 10.3622], [105.3682, 10.3542], [...",3.549591e+08,105.265754,10.421310
4,Feature,VNM.1.2_1,VNM,Vietnam,VNM.1_1,AnGiang,NA,ChợMới,ChoMoi,NA,Huyện,District,NA,VN.BD.AL,MultiPolygon,"[[[[105.4872, 10.5395], [105.4884, 10.5423], [...",3.685095e+08,105.461451,10.474941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,Feature,VNM.63.5_1,VNM,Vietnam,VNM.63_1,YênBái,NA,TrấnYên,TranYen,NA,Huyện,District,NA,VN.BN.YP,MultiPolygon,"[[[[104.8928, 21.5912], [104.8963, 21.5871], [...",6.275485e+08,104.789299,21.678250
706,Feature,VNM.63.6_1,VNM,Vietnam,VNM.63_1,YênBái,NA,VănChấn,VanChan,NA,Huyện,District,NA,VN.TQ.YS,MultiPolygon,"[[[[104.8726, 21.3489], [104.8684, 21.3536], [...",1.200524e+09,104.621486,21.565743
707,Feature,VNM.63.7_1,VNM,Vietnam,VNM.63_1,YênBái,NA,VănYên,VanYen,NA,Huyện,District,NA,VN.NA.YT,MultiPolygon,"[[[[104.7171, 21.7353], [104.7188, 21.7334], [...",1.395519e+09,104.552318,21.907291
708,Feature,VNM.63.8_1,VNM,Vietnam,VNM.63_1,YênBái,NA,YênBái,YenBai,NA,Thànhphố,City,NA,VN.BG.YT,MultiPolygon,"[[[[104.8725, 21.6812], [104.8738, 21.6798], [...",1.075042e+08,104.894701,21.707758


# Prepare the district code data

In [99]:
district_code = pd.read_csv('./large_files/district_code.csv')
def remove_administrative_terms(s):
    administrative_terms = ['Quận', 'Thành phố', 'Thành Phố', 'Huyện', 'Thị Xã', 'Tỉnh']
    for term in administrative_terms:
        s = re.sub(f'(?i){term}', '', s)
    return s.strip()

def merge_words(s):
    return s.replace(' ','')

dfs = []
for year in ['2010','2012','2014','2016']:
    relevant_year = [col for col in district_code.columns if col.endswith(year) and (col.startswith('dist') or col.startswith('provname'))]
    df = district_code[relevant_year].copy()
    df.loc[:,'year'] = year
    df.columns = ['provname','id','distname','year']
    dfs.append(df)

district_code = pd.concat(dfs, ignore_index=True,axis=0)
district_code.drop_duplicates(inplace=True)
district_code.dropna(inplace=True)

district_code['distname'] = district_code['distname'].apply(remove_administrative_terms)
district_code['provname'] = district_code['provname'].apply(remove_administrative_terms)

# Remove accent
district_code['distname'] = district_code['distname'].apply(unidecode)
district_code['provname'] = district_code['provname'].apply(unidecode)
# Remove spaces
district_code['distname'] = district_code['distname'].apply(merge_words)
district_code['provname'] = district_code['provname'].apply(merge_words)


C:\Users\lhmtr\AppData\Local\Temp\ipykernel_12188\3257561258.py:1: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  district_code = pd.read_csv('./large_files/district_code.csv')


# Merge all data together

In [101]:
gps['properties.VARNAME_1'] = gps['properties.NAME_1'].apply(remove_administrative_terms)
gps['properties.VARNAME_1'] = gps['properties.NAME_1'].apply(unidecode)

In [102]:
merged_df = district_code.merge(gps[['properties.VARNAME_1','properties.VARNAME_2','area','centroid_lon','centroid_lat']], how='left', left_on=['provname','distname'], right_on=['properties.VARNAME_1','properties.VARNAME_2'])

In [107]:
merged_df.drop(['properties.VARNAME_1','properties.VARNAME_2'],inplace=True,axis=1)

In [108]:
merged_df.to_csv('../data/district_data.csv')